In [1]:
from pathlib import Path
from topagnps2cche1d import Watershed

# root_dir = Path("../data_inputs/goodwin_creek_1m/")
root_dir = Path(
    "../data_inputs/goodwin_creek_1m_1_and_3_reaches/TopAGNPS_DataSets/3_Reach"
)
# root_dir = Path("../data_inputs/topagnps_ohio_files/")

path_to_reaches = root_dir / "AnnAGNPS_Reach_Data_Section.csv"
path_to_cells = root_dir / "AnnAGNPS_Cell_Data_Section.csv"
path_to_reaches_raster = root_dir / "AnnAGNPS_Reach_IDs.asc"

In [25]:
watershed = Watershed()
watershed.import_topagnps_reaches_network(path_to_reaches)
watershed.import_topagnps_cells(path_to_cells)

watershed.read_reaches_geometry_from_topagnps_asc_file(path_to_reaches_raster)

watershed.assign_strahler_number_to_reaches()
watershed.ignore_reaches_with_strahler_leq(0)

watershed.update_junctions_and_node_types()
watershed.identify_inflow_sources()

watershed.renumber_all_nodes_and_reaches_in_CCHE1D_computational_order()
watershed.set_node_id_to_compute_id()

watershed.update_watershed()

watershed.resample_reaches(id_list="all", numpoints=5)

Reading Reaches: 100%|██████████| 3/3 [00:00<00:00, 73.07it/s]


In [27]:
reaches = watershed.reaches
for reach in reaches.values():
    # if not v.ignore:
    print(reach)
# break

-----------------------------
Reach            : 2
CCHE1D Ch. ID    : 3
Receiving Reach  : 1
Slope            : 0.01716
Strahler Number  : 2
Ignore?          : False
(US/DS) Nodes    : (13, 17)
-----------------------------
Reach            : 3
CCHE1D Ch. ID    : 2
Receiving Reach  : 2
Slope            : 0.01082
Strahler Number  : 1
Ignore?          : False
(US/DS) Nodes    : (7, 12)
-----------------------------
Reach            : 4
CCHE1D Ch. ID    : 1
Receiving Reach  : 2
Slope            : 0.01332
Strahler Number  : 1
Ignore?          : False
(US/DS) Nodes    : (1, 6)


In [24]:
reaches = watershed.reaches
for reach in reaches.values():
    if reach.ignore:
        continue
    print("-----------------------------------------")
    print(f"Reach ID: {reach.id}")
    for node in reach.nodes.values():
        if node.inflow_cell_source is not None or node.inflow_reaches_source:
            # print(node)
            out_str = [
                f"-----------------------------------------",
                f"Node ID       : {node.id}",
                f"TYPE          : {node.type}",
                f"Cell Source   : {node.inflow_cell_source}",
                f"Reaches Source: {node.inflow_reaches_source}",
            ]
            print("\n".join(out_str))
# break

-----------------------------------------
Reach ID: 2
-----------------------------------------
Node ID       : 1
TYPE          : 1
Cell Source   : None
Reaches Source: [3, 4]
-----------------------------------------
Node ID       : 5
TYPE          : 9
Cell Source   : 23
Reaches Source: []


In [6]:
reach2 = watershed.reaches[2]
reach3 = watershed.reaches[3]
reach4 = watershed.reaches[4]

In [7]:
print(reach2.nodes[reach2.us_nd_id])

-----------------------------
Node ID         : 1
TYPE            : 1
USID            : None
DSID            : 2
US2ID           : None
COMPUTEID       : 1
Cell Source     : None
Reach Sources   : [3, 4]
(x,y)           : (240167.114013, 3795272.620724)
(row,col)       : (None, None)


In [8]:
print(reach3.nodes[reach3.ds_nd_id])

-----------------------------
Node ID         : None
TYPE            : 6
USID            : None
DSID            : None
US2ID           : None
COMPUTEID       : None
Cell Source     : 33
Reach Sources   : []
(x,y)           : (240168.114013, 3795273.620724)
(row,col)       : (226, 168)


In [9]:
print(reach4.nodes[reach4.ds_nd_id])

-----------------------------
Node ID         : None
TYPE            : 6
USID            : None
DSID            : None
US2ID           : None
COMPUTEID       : None
Cell Source     : 43
Reach Sources   : []
(x,y)           : (240454.114013, 3795214.620724)
(row,col)       : (285, 454)


## Plotting network

In [10]:
watershed.plot(
    frame_height=700, frame_width=900, line_width=2, by="CCHE1D_ID", title="Watershed"
)

:Overlay
   .NdOverlay.I  :NdOverlay   [CCHE1D_ID]
      :Curve   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)
   .NdOverlay.II :NdOverlay   [CCHE1D_ID]
      :Scatter   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID,CCHE1D_ID,Reach_ID)

In [11]:
for cell in watershed.cells.values():
    print(cell)

-----------------------------
Cell ID            : 22
Area (m²)          : 326.00
Receiving Reach ID : 2
Type               : right
-----------------------------
Cell ID            : 23
Area (m²)          : 479.00
Receiving Reach ID : 2
Type               : left
-----------------------------
Cell ID            : 31
Area (m²)          : 25010.00
Receiving Reach ID : 3
Type               : source
-----------------------------
Cell ID            : 32
Area (m²)          : 4248.00
Receiving Reach ID : 3
Type               : right
-----------------------------
Cell ID            : 33
Area (m²)          : 1429.00
Receiving Reach ID : 3
Type               : left
-----------------------------
Cell ID            : 41
Area (m²)          : 25002.00
Receiving Reach ID : 4
Type               : source
-----------------------------
Cell ID            : 42
Area (m²)          : 34881.00
Receiving Reach ID : 4
Type               : right
-----------------------------
Cell ID            : 43
Area (m²)     

In [12]:
print(reach3.nodes[reach3.ds_nd_id])

-----------------------------
Node ID         : None
TYPE            : 6
USID            : None
DSID            : None
US2ID           : None
COMPUTEID       : None
Cell Source     : 33
Reach Sources   : []
(x,y)           : (240168.114013, 3795273.620724)
(row,col)       : (226, 168)


In [13]:
import holoviews as hv
import numpy as np
from scipy.interpolate import make_interp_spline
from math import floor, ceil

In [14]:
def interpolate_xy_cord_step_numpoints(x, y, **kwargs):
    """
    Given two x and y arrays provides two new arrays xnew, ynew that are interpolated:
    key-value arguments:
        - step : define a step length (in the units of x and y) to resample points along the cord length
        OR
        - numpoints: define an arbitrary number of points to resample along the cord length
    WARNING: If both values are specified, the method yielding the greater of resampling nodes will be chosen
    It is advised to use only one of the keywords arguments.
    """
    if "step" in kwargs:
        step = kwargs["step"]
    else:
        step = None

    if "numpoints" in kwargs:
        numpoints = kwargs["numpoints"]
    else:
        numpoints = 0

    if step is None and numpoints == 0:
        raise Exception("No resampling parameter provided (step, or numpoints)")

    # Arrange coordinates in pairs:
    p = np.stack((x, y))

    dp = p[:, 1:] - p[:, :-1]  # 2 vector distance between points
    l = (dp**2).sum(axis=0)  # squares of lengths of 2-vectors between points
    u_cord = np.sqrt(l).cumsum()  # cumulative sums of 2-norms
    u_cord = np.r_[0, u_cord]  # the first point is parameterized at zero

    # Get arc length
    arc_length = u_cord[-1]  # by definition

    # Compute number of nodes needed for interpolation
    if step is not None:
        numpoints = max(
            ceil(arc_length / step), numpoints
        )  # chooses the highest value between
        # provided numpoints and the one computed
        # with the step method
    # create spline interpolant
    spl_cord = make_interp_spline(u_cord, np.c_[x, y])

    uu = np.linspace(u_cord[0], u_cord[-1], numpoints)
    xx_cord, yy_cord = spl_cord(uu).T

    return xx_cord, yy_cord

In [15]:
x, y = reach2.get_x_y_node_arrays_us_ds_order()

xx_cord, yy_cord = interpolate_xy_cord_step_numpoints(x, y, numpoints=21)

In [16]:
scatter_line_cord = hv.Scatter((xx_cord, yy_cord), label="cord").opts(
    aspect="equal", size=8
) * hv.Curve((xx_cord, yy_cord))
# scatter_line_c = (hv.Scatter((xx_c, yy_c), label='centripetal').opts(aspect='equal', size=8) * hv.Curve((xx_c,yy_c)))

# scatter_line_og

# scatter_line_c * scatter_line_cord

# (scatter_line_og * scatter_line_cord * scatter_line_c).opts(tools=['hover'])
(scatter_line_og * scatter_line_cord).opts(tools=["hover"])

NameError: name 'scatter_line_og' is not defined